In [1]:
from hypster import HP, config

In [2]:
@config
def rag_config(hp: HP):
    from haystack import Pipeline
    from haystack.components.builders import PromptBuilder
    from haystack.components.embedders import SentenceTransformersDocumentEmbedder, SentenceTransformersTextEmbedder
    from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
    from haystack.components.writers import DocumentWriter
    from haystack.document_stores.in_memory import InMemoryDocumentStore
    from haystack.document_stores.types import DuplicatePolicy

    document_store = InMemoryDocumentStore()
    model_types = ["sentence-transformers/all-MiniLM-L12-v2", "sentence-transformers/paraphrase-MiniLM-L6-v2"]
    emb_model = hp.select(model_types, default="sentence-transformers/all-MiniLM-L12-v2")

    document_writer = DocumentWriter(document_store, policy=DuplicatePolicy.OVERWRITE)
    doc_embedder = SentenceTransformersDocumentEmbedder(model=emb_model, batch_size=hp.int(16, min=2, max=256))

    indexing_pipeline = Pipeline()
    indexing_pipeline.add_component("document_writer", document_writer)
    indexing_pipeline.add_component("doc_embedder", doc_embedder)
    indexing_pipeline.connect("doc_embedder", "document_writer")

    text_embedder = SentenceTransformersTextEmbedder(model=emb_model)
    retriever = InMemoryEmbeddingRetriever(document_store, top_k=hp.int(10))

    temperature = hp.number(0, min=0, max=1, name="temperature")
    llm_provider = hp.select(["openai", "ollama"], default="openai", name="llm_provider")
    if llm_provider == "openai":
        from haystack.components.generators import OpenAIGenerator

        generator = OpenAIGenerator(
            model=hp.select(["gpt-4o", "gpt-4o-mini"], default="gpt-4o-mini", name="openai_model"),
            generation_kwargs={"temperature": temperature},
        )
    else:
        from haystack_integrations.components.generators.ollama import OllamaGenerator

        generator = OllamaGenerator(
            model=hp.select(
                {"llama3": "llama3", "llama3.1": "llama3.1", "mistral": "mistral"},
                default="llama3",
                name="ollama_model",
            ),
            url=hp.text("http://localhost:11434", name="ollama_base_url"),
            generation_kwargs={"temperature": temperature},
        )

    template = """
    Given the following information, answer the question.

    Context:
    {% for document in documents %}
        {{ document.content }}
    {% endfor %}

    Question: {{question}}
    Answer:
    """

    prompt_builder = PromptBuilder(template=template)

    rag_pipeline = Pipeline()

    rag_pipeline.add_component("text_embedder", text_embedder)
    rag_pipeline.add_component("retriever", retriever)
    rag_pipeline.add_component("prompt_builder", prompt_builder)
    rag_pipeline.add_component("llm", generator)

    rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
    rag_pipeline.connect("retriever", "prompt_builder.documents")
    rag_pipeline.connect("prompt_builder", "llm")


In [3]:
results = rag_config(values={"temperature": 0.5})

In [4]:
from hypster.ui import interactive_config

results = interactive_config(rag_config)

## Indexing

In [5]:
from datasets import load_dataset
from haystack import Document

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]
indexing_pipeline = results["indexing_pipeline"]
indexing_pipeline.warm_up()

indexing_pipeline.run({"doc_embedder": {"documents": docs}})

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

{'document_writer': {'documents_written': 151}}

## Response

In [6]:
rag_pipeline = results["rag_pipeline"]
question = "What does Rhodes Statue look like?"

response = rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Colossus of Rhodes was a statue of the Greek sun-god Helios, erected in the city of Rhodes. While scholars are not entirely sure of its exact appearance, they have a general idea based on contemporary depictions and descriptions. The statue is believed to have stood approximately 33 meters (108 feet) tall, making it one of the tallest statues of the ancient world.

The head of the statue likely featured curly hair with bronze or silver spikes radiating outward, resembling flames. This style was consistent with the standard renderings of the time. The statue was constructed with iron tie bars and brass plates forming its skin, while the interior was filled with stone blocks. It stood on a 15-meter-high (49-foot) white marble pedestal near the entrance to the Rhodes harbor. Overall, the Colossus was designed to celebrate the successful defense of Rhodes and was a symbol of freedom and independence for the city.
